In [ ]:
!pip install -U plotly

In [ ]:
!conda install datashader -y

In [ ]:
!conda install -c pyviz holoviews bokeh -y

In [ ]:
import pandas as pd
from os import listdir
from pandasql import sqldf
import matplotlib as plt
import plotly.express as px
from datetime import timedelta

In [ ]:
#definindo o diretorio
directory = 'data/'
files = listdir(directory)

In [ ]:
#selecionando apenas as variaveis importantes para esta analise
col_names = ['vendor_id', 'pickup_datetime','dropoff_datetime', 'pickup_longitude','pickup_latitude', 
             'dropoff_longitude', 'dropoff_latitude']

#setando os tipos para as variaveis, para que o pandas não precise adivinhar os dados e consumir mais RAM
dtypes = {
    "vendor_id": "category",
    "pickup_datetime": "datetime64",
    "dropoff_datetime": "datetime64",
    "pickup_longitude": "float",
    "pickup_latitude": "float",
    "dropoff_longitude":"float",
    "dropoff_latitude": "float"
}

#criando um dataframe vazio apenas para as colunas. Este dataframe servira para concatenar todos os datasets
#posteriormente
df = pd.DataFrame(columns=col_names)

#um for que le apenas os .json
for file_name in files:
    if '.json' in file_name:
        #le o primeiro json tranformando-o em dataframe do pandas
        df_new = pd.read_json(directory+file_name, lines=True, dtype=dtypes)
        #pega apenas as colunas que serao importantes para as analises e sobreescreve o dataframe
        df_new = df_new[col_names]

        #concatena o dataframe antigo com o novo
        df = pd.concat([df, df_new])

df = df.reset_index(drop=True)

### Tempo médio das corridas nos dias de sábado e domingo

In [ ]:
#tempo medio das curridas durante a semana
df_workdays = df[df['pickup_datetime'].dt.weekday < 5]

seconds = df_workdays['dropoff_datetime'].sub(df_workdays['pickup_datetime']).mean().seconds 
str(timedelta(seconds=seconds))

In [ ]:
#tempo medio das corridas nos finais de semana
df_weekends = df[(df['pickup_datetime'].dt.weekday == 5) | (df['pickup_datetime'].dt.weekday == 6)]

seconds = df_weekends['dropoff_datetime'].sub(df_weekends['pickup_datetime']).mean().seconds 
str(timedelta(seconds=seconds))

### Mapa com latitude e longitude de pickups and dropoffs no ano de 2010

#### Pickups

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

In [ ]:
#filtrando dados com ano 2010
df_2010 = df[df['pickup_datetime'].dt.year == 2010]

In [ ]:
#delimitando o mapa de New York para remover coordenadas muito distantes (outliers e coordenadas erradas)
df_pickup = df_2010.query('pickup_latitude < 40.90').query('pickup_latitude > 40.50').query('pickup_longitude > -74.25').query('pickup_longitude < -73.82')

In [ ]:
#cria um canvas vazio para o tamanho da imagem
cvs_pickup = ds.Canvas(plot_width=1000, plot_height=1000)

#adiciona os pontos no canvas baseado na latitude e longitude
agg_pickup = cvs_pickup.points(df_pickup, x='pickup_longitude', y='pickup_latitude')

#pega as coordenadas para usarmos posteriormente no mapa do plotly
coords_lat_pickup, coords_lon_pickup = agg_pickup.coords['pickup_latitude'].values, agg_pickup.coords['pickup_longitude'].values

coordinates_pickup = [[coords_lon_pickup[0], coords_lat_pickup[0]],
               [coords_lon_pickup[-1], coords_lat_pickup[0]],
               [coords_lon_pickup[-1], coords_lat_pickup[-1]],
               [coords_lon_pickup[0], coords_lat_pickup[-1]]]

In [ ]:
#cria uma imagem baseada no canvas do passo anterior
img_pickup = tf.shade(agg_pickup, cmap=['yellow', 'red'], how='log')[::-1].to_pil()

In [ ]:
#pega a primeira coordenada do dataset para poder dar zoom no mapa ao plotar
fig = px.scatter_mapbox(df_pickup[:1], lat='pickup_latitude', lon='pickup_longitude', zoom=10.5)

#atualiza o layout adicionado a imagem com os pontos com a referencia das coordenadas
fig.update_layout(mapbox_style="carto-darkmatter",
                 mapbox_layers = [
                {
                    "sourcetype": "image",
                    "source": img_pickup,
                    "coordinates": coordinates_pickup
                }]
)
fig.show()
#fig.write_image("graficos/pickups_maps.jpeg")

#### Dropoffs

In [ ]:
#delimitando o mapa de New York para remover coordenadas muito distantes
df_dropoff = df_2010.query('dropoff_latitude < 40.90').query('dropoff_latitude > 40.50').query('dropoff_longitude > -74.25').query('dropoff_longitude < -73.82')

In [ ]:
#cria um canvas vazio para o tamanho da imagem
cvs_dropoff = ds.Canvas(plot_width=1000, plot_height=1000)

#adiciona os pontos no canvas baseado na latitude e longitude
agg_dropoff = cvs_dropoff.points(df_dropoff, x='dropoff_longitude', y='dropoff_latitude')

#pega as coordenadas para usarmos posteriormente no mapa do plotly
coords_lat_dropoff, coords_lon_dropoff = agg_dropoff.coords['dropoff_latitude'].values, agg_dropoff.coords['dropoff_longitude'].values

coordinates_dropoff = [[coords_lon_dropoff[0], coords_lat_dropoff[0]],
               [coords_lon_dropoff[-1], coords_lat_dropoff[0]],
               [coords_lon_dropoff[-1], coords_lat_dropoff[-1]],
               [coords_lon_dropoff[0], coords_lat_dropoff[-1]]]

In [ ]:
#cria uma imagem baseada no canvas do passo anterior
img_dropoff = tf.shade(agg_dropoff, cmap=['lightblue', 'green'], how='log')[::-1].to_pil()

In [ ]:
#pega a primeira coordenada do dataset para poder dar zoom no mapa ao plotar
fig = px.scatter_mapbox(df_pickup[:1], lat='dropoff_latitude', lon='dropoff_longitude', zoom=10.5)

#atualiza o layout adicionado a imagem com os pontos com a referencia das coordenadas
fig.update_layout(mapbox_style="carto-darkmatter",
                 mapbox_layers = [
                {
                    "sourcetype": "image",
                    "source": img_dropoff,
                    "coordinates": coordinates_dropoff
                },
                ]
)
fig.show()
#fig.write_html("graficos/dropoffs_maps.html")